[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/okada-tak/deep-learning-from-scratch-2/blob/master/notebooks/ch04.ipynb)

# 4章 word2vecの高速化 のまとめ
- Embeddedレイヤは単語の分散表現を格納し、順伝播において該当する単語IDのベクトルを抽出する
- word2vecでは語彙数の増加に比例して計算量が増加するので、近似計算を行う高速な手法を使うとよい
- Negative Samplingは負例をいくつかサンプリングする手法であり、これを利用すれば多値分類を二値分類として扱うことができる
- word2vecによって得られた単語の分散表現は、単語の意味が埋め込まれたものであり、似たコンテキストで使われる単語は単語ベクトルの空間上で近い場所に位置するようになる
- word2vecの単語の分散表現は、類推問題をベクトルの加算と減算によって解ける性質を持つ
- word2vecは転移学習の点で特に重要であり、その単語の分散表現はさまざまな自然言語処理のタスクに利用できる

## 4.1 word2vecの改良①
ボトルネック：  
- 入力層のone-hot表現と重み行列$\text{W}_\text{in}$の積による計算（4.1節で解決）
- 中間層と重み行列$\text{W}_\text{out}$の積およびSoftmaxレイヤの計算（4.2節で解決）



### 4.1.1 Embeddingレイヤ
重みパラメータから「単語IDに該当する行（ベクトル）」を抜き出すレイヤ：Embdddingレイヤ  
単語の分散表現が格納される

### 4.1.2 Embeddingレイヤの実装


In [ ]:
import numpy as np
W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [ ]:
W[2]

array([6, 7, 8])

In [ ]:
W[5]

array([15, 16, 17])

In [ ]:
idx = np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

common/layers.py

In [ ]:
# p.136-137
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

# p.138
#    def backward(self, dout):
#        dW, = self.grads
#        dW[...] = 0
#        dW[self.idx] = dout     # 悪い例
#        return None

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
#        if GPU:
#            np.scatter_add(dW, self.idx, dout)
#        else:
        np.add.at(dW, self.idx, dout)
        return None

## 4.2 word2vecの改良②
Negative Samplingを導入する

### 4.2.1 中間層以降の計算の問題点
- 中間層のニューロンと重み行列$\text{W}_\text{out}$の積
- Softmaxレイヤの計算

今回の例でのSoftmaxの式：  
$$
y_k=\frac{\exp(s_k)}{\displaystyle \sum_{i=1}^{1000000}\exp(s_i)}
$$

### 4.2.2 多値分類から二値分類へ
「コンテキストが『you』と『goodbye』のとき、ターゲットとなる単語は何ですか？」  
↓  
「コンテキストが『you』と『goodbye』のとき、ターゲットとなる単語は『say』ですか？」  

出力側の重み$\text{W}_\text{out}$では、各単語IDの単語ベクトルが各列に格納されている。  
『say』の単語ベクトルを抽出し、中間層のニューロンとの内積を求める。  
$$
(1 \times 100) \cdot (100 \times 1) = (1 \times 1)
$$

### 4.2.3 シグモイド関数と交差エントロピー誤差
二値分類：スコアにシグモイド関数を適用して確率を得る。損失関数は「交差エントロピー誤差」  
（多値分類：スコアにソフトマックス関数を適用して確率を得る。損失関数は同じく「交差エントロピー誤差」）  

### 4.2.4 多値分類から二値分類へ（実装編）
Embeddingレイヤと「dot演算（内積）」の処理を合わせたEmbedding Dotレイヤを導入する。  


## ■Colaboratory用
Google Colaboratoryの場合、Google Driveに  
dl-from-scratch-2/ch04  
というフォルダを用意し、そこにこのjupyter notebookを配置。  
(dl-from-scratch-2の部分は任意。)  
また、datasetフォルダとcommonフォルダを
dl-from-scratch-2/dataset  
dl-from-scratch-2/common
にコピーしておく。  

以下のセルでGoogle Driveをマウント。許可を求められるので許可する。

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## ■Colaboratory用
chdirする。

In [10]:
import sys,os
os.chdir('/content/drive/My Drive/dl-from-scratch-2/ch04')
os.getcwd()

'/content/drive/My Drive/dl-from-scratch-2/ch04'

ch04/negative_sampling_layer.py

In [ ]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections


class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 4.2.5 Negative Sampling
ネガティブな例（負例）を少数サンプリングして学習する。  
正例とサンプリングされた負例における損失を足し合わせて最終的な損失とする。

### 4.2.6 Negative Samplingのサンプリング手法


In [1]:
import numpy as np

np.random.choice(10)

3

In [2]:
# もう1回
np.random.choice(10)

4

In [4]:
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'I'

In [5]:
np.random.choice(words, size=5)

array(['say', 'say', 'you', 'hello', 'goodbye'], dtype='<U7')

In [6]:
np.random.choice(words, size=5, replace=False)

array(['I', 'hello', '.', 'goodbye', 'say'], dtype='<U7')

In [7]:
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

Negative samplingの式：
$$
P'(w_i)=\frac{P(w_i)^{0.75}}{\displaystyle \sum_{j}^{n}P(w_j)^{0.75}}
$$

In [8]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /=np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


ch04/negative_sampling_layer.py

In [11]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）で計算するときは、速度を優先
            # 負例にターゲットが含まれるケースがある
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [12]:
corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[2 3]
 [0 2]
 [2 1]]


### 4.2.7 Negative Samplingの実装


ch04/negative_sampling_layer.py

In [ ]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 正例のフォワード
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 負例のフォワード
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

## 4.3 改良版word2vecの学習
### 4.3.1 CBOWモデルの実装

ch04/cbow.py

In [13]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # レイヤの生成
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embeddingレイヤを使用
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # すべての重みと勾配をリストにまとめる
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

### 4.3.2 CBOWモデルの学習コード


ch04/train.py  
長時間かかるようなのでとりあえずパス。

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common import config
# GPUで実行する場合は、下記のコメントアウトを消去（要cupy）
# ===============================================
# config.GPU = True
# ===============================================
from common.np import *
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# ハイパーパラメータの設定
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# モデルなどの生成
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 学習開始
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 後ほど利用できるように、必要なデータを保存
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

### 4.3.3 CBOWモデルの評価

ch04/eval.py

In [14]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# most similar task
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


common/util.py

In [17]:
def analogy(a, b, c, word_to_id, id_to_word, word_matrix, top=5, answer=None):
    for word in (a, b, c):
        if word not in word_to_id:
            print('%s is not found' % word)
            return

    print('\n[analogy] ' + a + ':' + b + ' = ' + c + ':?')
    a_vec, b_vec, c_vec = word_matrix[word_to_id[a]], word_matrix[word_to_id[b]], word_matrix[word_to_id[c]]
    query_vec = b_vec - a_vec + c_vec
    query_vec = normalize(query_vec)

    similarity = np.dot(word_matrix, query_vec)

    if answer is not None:
        print("==>" + answer + ":" + str(np.dot(word_matrix[word_to_id[answer]], query_vec)))

    count = 0
    for i in (-1 * similarity).argsort():
        if np.isnan(similarity[i]):
            continue
        if id_to_word[i] in (a, b, c):
            continue
        print(' {0}: {1}'.format(id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

def normalize(x):
    if x.ndim == 2:
        s = np.sqrt((x * x).sum(1))
        x /= s.reshape((s.shape[0], 1))
    elif x.ndim == 1:
        s = np.sqrt((x * x).sum())
        x /= s
    return x

In [18]:
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)


[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875


## 4.4 word2vecに関する残りのテーマ
### 4.4.1 word2vecを使ったアプリケーションの例
転移学習

### 4.4.2 単語ベクトルの評価方法
類似性：人が出した類似度のスコアとword2vecによるコサイン類似度のスコアを比較して相関性をみる  
類推問題：「king : queen = man : ?」のような類推問題を出題し、その正解率をみる  

- モデルによって精度が異なる（コーパスに応じて最適なモデルを選ぶ）
- コーパスが大きいほど良い結果になる（ビッグデータは常に望まれる）
- 単語ベクトルの次元数は適度な大きさが必要（大きすぎても精度が悪くなる）

## 4.5 まとめ
